# SDOF system with energy dissipation based upon snab-back mechanisms

In [1]:
import numpy as np
from IPython.display import display
from scipy.integrate import odeint
from matplotlib import pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

## Simple sdof oscillator

Consider the following oscillator

$$M\ddot U(t)+C\dot U(t)+KU(t)=F_0S_{\omega t}$$

Perform a change of variables to convert the 2-nd order ODE into a system of first order ODEs:

$$\dot U(t)= V(t)$$

$$\dot V(t)=\frac1M \lbrack F_0S_{\omega t}-CV(t)-KU(t)\rbrack$$

and place these ODEs into a callable function:

## Misses truss with spring

In [2]:
def mis_dis(F_0 , fac , lmda):
#    F_0   = 10.0    #Force amplitude for the Misses truss
#    fac   = 1.0    #Piecewise continous at fac*lambda
#    lmda  = 0.5   #Wavelength for the Misses truss F-d relationship
#
    x  = np.linspace(-2*lmda , 2*lmda, 1001) #Assumed displacements span
    n = len(x)
    FM = np.zeros(n)
#
    for i in range(n):
        FM[i] = misses(F_0 , fac , lmda , x[i])
        
        
    fig3 = plt.figure(figsize=(12,6))
    axes3= fig3.add_axes([0.1,0.1,0.8,0.8] )
    axes3.set_ylabel('$U$')
    axes3.set_xlabel('$t[s]$')
    axes3.plot(x , FM )    
    plt.grid()
    return

In [3]:
def misses(F_0 , fac , lmda , x):
    """
    Constitutive law for the Misses truss, This is piecewise
    continuous function.
    F_0 : force amplitude
    fac : number of wavelengths at which the constant slope phase starts
    lmda: Wavelength
    x   : Displacement
    """
    dx = lmda/100
    df = F_0*np.sin((2*np.pi/lmda)*dx)
    k = df/dx
    if x > -fac*lmda and x < fac*lmda:
        F = F_0*np.sin((2*np.pi/lmda)*x)
    else:
        if x<= -fac*lmda:
            F = k*(x + fac*lmda)
        else:
            F = k*(x - fac*lmda)
    return F

In [4]:
w = interactive(mis_dis , F_0 = widgets.FloatSlider(min = 0 , max=100 , step =10 , value =50) , fac = (0.5 , 1.0) , lmda =(0.1 , 1.0))
w.children
display(w)

interactive(children=(FloatSlider(value=50.0, description='F_0', step=10.0), FloatSlider(value=0.75, descripti…

## System with  added dissipation device

In [5]:
def model(z , t , M , C , K , f_0 , ome):
    """
    z would be the solution vector
    dzdt stores the system of ODEs ready for integration.
    """
    U = z[0]
    V = z[1]
    f_t  = force(f_0 , ome , t)
    dUdt = V
    dVdt = (1/M)*(f_t - C*V - K*U)
    dzdt = [dUdt , dVdt]
    
    return dzdt

In [6]:
def force(f_0 , ome , t):
    """
    Harmonic force of frequency omega
    """
    f_t = f_0*np.sin(ome*t)
    return f_t

In [7]:
def dissip_device(z , t , M , C , K , f_0 , ome , m , c , k , F_0 , fac , lmda):
    """
    z would be the solution vector
    dzdt stores the system of ODEs ready for integration.
    """
    U = z[0]
    V = z[1]
    u = z[2]
    v = z[3]
#
    F_AP  = force(f_0 , ome , t)
    F_VM  = misses(F_0 , fac , lmda ,U-u)
#
    dUdt = V
    dVdt = (1/M)*(F_AP - C*V - K*U-F_VM)
    dudt = v
    dvdt = (1/m)*(F_VM - k*u - c*v)
#
    dzdt = [dUdt , dVdt , dudt , dvdt]
    
    return dzdt

In [8]:
def device_integrate(M , K , ome , C , f_0 , c , k , F_0 , fac , lmda):
    """
    M   :Mass
    C   :Dashpot coefficient
    K   :Stiffness
    f_0 :Amplitude of the applied force
    ome :Frequency of applied force
    
    """
    y0 = [0.0 , 0.0]                     # Initial conditions
    t    = np.linspace(0 , 20 , 1001)    #Time span
    sol  = odeint(model , y0 , t , args=(M , C , K , f_0 , ome))
    
    U_ref = sol[: , 0]
    V_ref = sol[: , 1]
        
    m   = 1.0e-3 

    y0   = [0.0 , 0.0 , 0.0 , 0.0]
    t    = np.linspace(0 , 20 , 1001)    #Time span
    sol  = odeint(dissip_device , y0 , t , args =(M , C , K , f_0 , ome , m , c , k , F_0 , fac , lmda))
    
    U_dev = sol[: , 0] 
    V_dev = sol[: , 1]
    u_dev = sol[: , 2]
    v_dev = sol[: , 3]
    F_r = K*U_dev + C*V_dev + k*u_dev + c*v_dev
    F_s = K*U_dev + k*u_dev
    F_v = C*V_dev + c*v_dev
#
    fig4 = plt.figure(figsize=(12,6))
    axes4= fig4.add_axes([0.1,0.1,0.8,0.8] )
    axes4.set_ylabel('$U$')
    axes4.set_xlabel('$t[s]$')
    axes4.plot(t, U_dev , 'r' , label='$U_{dev}$' )
    axes4.plot(t, U_ref , 'g' , label='$U_{ref}$' )
    plt.grid()
    
    fig5 = plt.figure(figsize=(12,6))
    axes5= fig5.add_axes([0.1,0.1,0.8,0.8] )
    axes5.set_ylabel('$F$')
    axes5.set_xlabel('$t[s]$')
    axes5.plot(U_dev , F_r )
    axes5.plot(U_dev , F_s )
    axes5.plot(U_dev , F_v)
    plt.grid()


        
    return

In [11]:
interact(device_integrate , M = (1, 100) , K = (10 , 300 , 10) , ome =(1 , 30) , C = (0.1 , 0.5) , f_0 = (1 , 20) ,
         c =(0.0 , 0.5) , k=(0 , 100 , 10) , F_0 = (1.0, 10.0) , fac = (0.5 , 1.0) , lmda =(0.1 , 1.0) )

interactive(children=(IntSlider(value=50, description='M', min=1), IntSlider(value=150, description='K', max=3…

<function __main__.device_integrate(M, K, ome, C, f_0, c, k, F_0, fac, lmda)>

In [10]:
from IPython.core.display import HTML
def css_styling():
    styles = open('./nb_style.css', 'r').read()
    return HTML(styles)
css_styling()